In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import koreanize_matplotlib

In [19]:
train_df = pd.read_csv('./data/ratings_train.txt', sep="\t")
train_df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [20]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [21]:
train_df = train_df.dropna()
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 149995 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        149995 non-null  int64 
 1   document  149995 non-null  object
 2   label     149995 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.6+ MB


In [22]:
train_df['label'].value_counts()

label
0    75170
1    74825
Name: count, dtype: int64

# 정규표현식을 사용한 텍스트 정리

In [23]:
import re

In [24]:
def text_clean(x):
    # 한글, 영문대소문자, 숫자만 남기고 모두 제거
    cleaned = re.sub(r'[^가-힣a-zA-Z0-9]', " ", x)
    cleaned = cleaned.replace("  ", " ").replace("  ", " ").strip()
    return cleaned

In [25]:
train_df.loc[:, 'document'] = train_df['document'].apply(text_clean)

In [26]:
X = train_df['document']
y = train_df['label']

In [27]:
X

0                                         아 더빙 진짜 짜증나네요 목소리
1                              흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나
2                                         너무재밓었다그래서보는것을추천한다
3                                교도소 이야기구먼 솔직히 재미는 없다 평점 조정
4         사이몬페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어보이기만 했던 커스틴 ...
                                ...                        
149995                                      인간이 문제지 소는 뭔죄인가
149996                                           평점이 너무 낮아서
149997                        이게 뭐요 한국인은 거들먹거리고 필리핀 혼혈은 착하다
149998                          청춘 영화의 최고봉 방황과 우울했던 날들의 자화상
149999                             한국 영화 최초로 수간하는 내용이 담긴 영화
Name: document, Length: 149995, dtype: object

In [28]:
y

0         0
1         1
2         0
3         0
4         1
         ..
149995    0
149996    1
149997    0
149998    1
149999    0
Name: label, Length: 149995, dtype: int64

# 훈련 데이터와 테스트 데이터로 분리 (홀드아웃)

In [29]:
from sklearn.model_selection import train_test_split

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=10, stratify=y)

# 문자를 벡터화 countvectorizer()
- 모든 문자를 숫자로 변환해서 단어 사전을 만들어 줌

In [31]:
from sklearn.feature_extraction.text import CountVectorizer

In [32]:
doc = ['고양이가 나무 위에 있다', '나무 아래에 고양이가 있다']

In [33]:
c_vec = CountVectorizer()
x = c_vec.fit_transform(doc)
print(c_vec.get_feature_names_out()) # 벡터화 된 단어 목록
print(x.toarray()) # 단어의 빈도 벡터 출력

['고양이가' '나무' '아래에' '위에' '있다']
[[1 1 0 1 1]
 [1 1 1 0 1]]


In [34]:
cv = CountVectorizer()
cv.fit(X_train)
X_train_cv = cv.transform(X_train)
X_test_cv = cv.transform(X_test)

# 나이브베이즈로 감성분석 하기
- MultinomialNB

In [35]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

In [36]:
mnb = MultinomialNB()
mnb.fit(X_train_cv, y_train)
pred = mnb.predict(X_test_cv)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.81      0.84      0.82     30068
           1       0.83      0.80      0.82     29930

    accuracy                           0.82     59998
   macro avg       0.82      0.82      0.82     59998
weighted avg       0.82      0.82      0.82     59998



# RandomForest와 비교

In [37]:
from sklearn.ensemble import RandomForestClassifier

In [38]:
%%time
rfc = RandomForestClassifier(n_estimators=300, n_jobs=6, random_state=10)
rfc.fit(X_train_cv, y_train)
pred2 = rfc.predict(X_test_cv)
print(classification_report(y_test, pred2))

              precision    recall  f1-score   support

           0       0.79      0.75      0.77     30068
           1       0.76      0.80      0.78     29930

    accuracy                           0.77     59998
   macro avg       0.78      0.77      0.77     59998
weighted avg       0.78      0.77      0.77     59998

CPU times: user 2h 44min 22s, sys: 7.13 s, total: 2h 44min 29s
Wall time: 27min 29s


짱 오래 걸림

근데 CounterVectorizer는 띄어쓰기 기준 -> 한국어에는 적합하지는 않음 <br>
한국어에는 조사가 있어서 형태소 분리를 해줘야 올바르게 분석할 수 있음

# 한국어는 반드시 형태소 분석기로 형태소를 나눈 후 분석해야 한다.
- morphs: 형태소를 분리해서 단어만 출력
- pos: 형태소를 분리해서 단어와 형태소 종류를 튜플로 같이 출력
- nouns: 형태소를 분리해서 명사만 출력

In [ ]:
# mecab을 이용한 형태소 분리
from konlpy.tag import Mecab
mecab = Mecab()

In [ ]:
text = "한국어는 반드시 형태소 분석기로 형태소를 나눈 후 분석해야 한다."
print(mecab.morphs(text))
print(mecab.nouns(text))
print(mecab.pos(text))

# konlpy의 Mecab을 이용해 형태소 분리

In [ ]:
def tokenizer(text):
    tokens = mecab.morphs(text)
    return tokens

# CountVectorizer, TfidfVectorizer
### 1) CountVectorizer: 단어 빈도(횟수) 기반 벡터화
- 작동 방식: CountVectorizer는 각 문서에서 등장한 단어의 횟수(빈도)를 기준으로 벡터를 만듭니다. 단순히 각 문서에서 특정 단어가 몇 번 등장했는지를 세어 벡터화합니다.<br>
- 특징:
    - 각 단어의 빈도가 높을수록 해당 단어의 중요성이 더 크다고 가정합니다.<br>
    - 빈도가 높다는 것만을 고려하기 때문에, 모든 문서에서 자주 등장하는 단어도 중요한 단어로 처리될 수 있습니다.<br>
- 예시:<br>
    - 예를 들어, 두 개의 문서가 있다면:
        ```
        문서 1: "고양이가 나무 위에 있다."
        문서 2: "나무 아래에 고양이가 있다." 
        ```
    - 이 두 문서를 CountVectorizer로 변환하면 단어 빈도가 포함된 벡터가 생성됩니다:<br>
    - `['고양이': 2, '나무': 2, '위에': 1, '아래에': 1, '있다': 2]`
### 2) TfidfVectorizer: TF-IDF (Term Frequency-Inverse Document Frequency) 기반 벡터화
- 작동 방식: TfidfVectorizer는 단어의 빈도뿐만 아니라, 단어의 중요도를 계산합니다. 여기서는 TF-IDF 값을 사용하여 문서 간 차별성을 강조합니다.<br>
- TF (Term Frequency): 단어가 문서에서 얼마나 자주 등장했는지를 나타냅니다.<br>
- IDF (Inverse Document Frequency): 단어가 다른 문서에 얼마나 자주 등장하지 않았는지를 나타냅니다. 자주 등장하지 않는 단어는 더 중요한 단어로 간주합니다.<br>
- 특징:<br>
    - TF-IDF는 문서 전체에서 자주 등장하는 흔한 단어들(예: "그리고", "이다" 등)의 중요도를 낮추고, 문서에서만 중요한 단어들의 중요도를 높입니다.<br>
    - 단순히 빈도가 높은 단어보다 특정 문서에서 더 특징적인 단어에 더 높은 가중치를 부여합니다.<br>
- 예시:<br>
    - 위의 문서 1과 문서 2에 대해 TfidfVectorizer로 변환하면, 공통 단어들(예: "있다", "고양이")의 중요도는 낮아지고, 차별적인 단어(예: "위에", "아래에")의 중요도는 상대적으로 높아집니다.


CountVectorizer와 TfidfVectorizer 비교
| 특성             | CountVectorizer                                     | TfidfVectorizer                                               |
|----------------------|--------------------------------------------------------|-------------------------------------------------------------------|
| 기반             | 단어의 단순 빈도                                        | 단어 빈도 + 문서 내에서의 상대적 중요도(TF-IDF)                     |
| 단어 빈도 계산    | 문서에서 등장한 단어의 단순한 등장 횟수를 셈            | 단어의 등장 횟수(TF)와 해당 단어가 문서들에서 얼마나 자주 등장하지 않았는지를 함께 고려(IDF) |
| 빈번한 단어 처리  | 문서에서 자주 등장하는 단어일수록 높은 가중치를 부여    | 문서에서 흔한 단어는 가중치를 낮추고, 드문 단어는 높은 가중치를 부여   |
| 주요 용도        | 단순한 단어 빈도 기반 분석이 필요할 때 사용             | 문서 간 차별적인 단어를 구별할 때 유용                                |
| 계산 비용         | 상대적으로 적음                                         | 상대적으로 더 복잡하고 계산 비용이 높음                                |

- tokenizer: 형태소를 분석해서 단어를 나누어 주는 역할, 기본은 공백을 기준으로 나눔
    - 한국어는 조사가 있기 때문에 공백이 아닌 형태소 분석을 통해서 나눠야 함.
    - konlpy의 형태소 분석기를 이용해서 형태소를 나누고 분석
- ngram_range(1, 2)
    - 단어를 벡터화 할 때 단어의 범위를 지정 (1, 2)는 1-gram, 2-gram
    - "이 영화는 정말 좋다"
    - 1-gram: ['이', '영화는', '정말', '좋다']
    - 2-gram: ['이 영화는', '영화는 정말', '정말 좋다']
- min_df=4 
    - 단어가 등장하는 최소 문서 수를 설정하는 파라미터
- max_df=0.9
    - 단어가 전체 문서의 90% 이하에서 등장할 때만 벡터화에 포함

# 한국어 형태소 분석, Countvectorizer/ Tfidfvectorizer 비교

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=10, stratify=y)

In [ ]:
# countvectorizer + mecab
cv_mecab = CountVectorizer(tokenizer=tokenizer, token_pattern=None, ngram_range=(1,2), min_df=4, max_df=0.9)
cv_mecab.fit(X_train)
cv_mecab_X_train = cv_mecab.transform(X_train)
cv_mecab_X_test = cv_mecab.transform(X_test)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# countvectorizer + mecab
tfidf_mecab = TfidfVectorizer(tokenizer=tokenizer, token_pattern=None, ngram_range=(1,2), min_df=4, max_df=0.9)
tfidf_mecab.fit(X_train)
tfidf_mecab_X_train = tfidf_mecab.transform(X_train)
tfidf_mecab_X_test = tfidf_mecab.transform(X_test)

# countvectorizer + mecab 분석

In [ ]:
mnb1 = MultinomialNB()
mnb1.fit(cv_mecab_X_train, y_train)
pred1 = mnb1.predict(cv_mecab_X_test)
print(classification_report(y_test, pred1))

# TF-IDFvectorizer + mecab 분석

In [ ]:
mnb2 = MultinomialNB()
mnb2.fit(tfidf_mecab_X_train, y_train)
pred2 = mnb2.predict(tfidf_mecab_X_test)
print(classification_report(y_test, pred2))

# 만들어진 모델 내보내기
- 데이터를 분석해서 모델을 만들고 서비스에서 사용하기 위해서는 아래의 사항이 동일해야 한다.
    1. 전처리 과정, 결측값, 이상값 처리, 원핫인코딩 등
    2. 스케일링, 원핫인코딩, 벡터화 했을 때 정보를 반드시 서비스 코드로 이동
    3. 모델도 저장해서 서비스 코드로 이동
- joblib을 통해서 직렬화 후 내보내기/불러오기 가능

In [ ]:
#!pip install joblib

In [ ]:
import joblib

In [ ]:
joblib.dump(tokenizer, "./data/tokenizer.joblib")

In [ ]:
joblib.dump(cv_mecab, "./data/cv_mecab.joblib")

In [ ]:
joblib.dump(tfidf_mecab, "./data/tfidf_mecab.joblib")

In [ ]:
joblib.dump(mnb1, "./data/mnb1_model.joblib")

In [ ]:
joblib.dump(mnb2, "./data/mnb2_model.joblib")

In [ ]:
joblib.dump(text_clean, "./data/text_clean.joblib")